In [1]:
using LinearAlgebra
using BlackBoxOptim
using HDF5

Fidelity for qubit can be written:

$F(\rho, \sigma) = \operatorname{tr}(\rho \sigma) + 2\sqrt{\det(\rho) \det(\sigma)}$

In [2]:
function infidelity_norm(ρ, σ)
    real(1 - tr(ρ * σ) - 2*sqrt(det(ρ)*det(σ)))
end

infidelity_norm (generic function with 1 method)

In [3]:
function read_timeevolution(file_name, state, γ)
    h5open(file_name, "r") do file
        ρᵧ = read(file[state][string(γ)])
        t = ρᵧ["t"]
        ρ₀₀ = ρᵧ["p0"]; Re_ρ₀₁ = ρᵧ["s_re"];  Im_ρ₀₁ = ρᵧ["s_im"]
        ρ_series = []
        t_series = []

        for i in 1:length(t)
            ρᵢ= [ ρ₀₀[i]                      Re_ρ₀₁[i] + im * Im_ρ₀₁[i]
                  Re_ρ₀₁[i] - im * Im_ρ₀₁[i]  1 - ρ₀₀[i]                 ]
            push!(ρ_series, convert(Matrix{ComplexF64}, ρᵢ))
            push!(t_series, convert(Float64, t[i]))
        end
        return(t_series, ρ_series)
    end
end

# Define the Pauli matrices
σˣ = [0 1; 1 0]
σʸ = [0 im; -im 0]
σᶻ = [1 0; 0 -1]

# Define the basis elements
fᴷ₁ = σˣ / 2
fᴷ₂ = σʸ / 2
fᴷ₃ = σᶻ / 2

# Check orthogonality and normalization
@assert tr(fᴷ₁ * fᴷ₂) ≈ 0
@assert tr(fᴷ₁ * fᴷ₃) ≈ 0
@assert tr(fᴷ₂ * fᴷ₃) ≈ 0
@assert tr(fᴷ₁ * fᴷ₁) ≈ 1 / 2
@assert tr(fᴷ₂ * fᴷ₂) ≈ 1 / 2
@assert tr(fᴷ₃ * fᴷ₃) ≈ 1 / 2

fᴼᴺᴮ = [fᴷ₁, fᴷ₂, fᴷ₃]

# Function to calculate Dc
function Dc(ρ, t, H, C)
    U = (H * ρ - ρ * H) / im
    D = sum(C .* [2 * fᵢ * ρ * fⱼ' - ρ * fⱼ' * fᵢ - fⱼ' * fᵢ * ρ for fᵢ in fᴼᴺᴮ, fⱼ in fᴼᴺᴮ]) / 2
    return U + D
end

function construct_H(params)
    ϵ, h_Re, h_Im = params[1:3]
    return [
        ϵ               h_Re + im * h_Im
        h_Re - im * h_Im -ϵ
    ] / 2
end

function construct_C(params)
    m_Re = reshape(params[4:12], (3, 3))
    m_Im = reshape(params[13:21], (3, 3))
    M = m_Re + im * m_Im
    return M * M'
end


# Define the objective function for optimization
function kossak_obj(params, ρ, t)
    H = construct_H(params)
    C = construct_C(params)

    obj = 0.0
    for i in 3:length(ρ)
        ρ1 = ρ[i]
        ρ2 = ρ[i - 2] + (t[i] - t[i - 1]) * (Dc(ρ[i], t[i], H, C) + 4 * Dc(ρ[i - 1], t[i - 1], H, C) + Dc(ρ[i - 2], t[i - 2], H, C)) / 3
        obj += infidelity_norm(ρ1,ρ2)
    end
    return obj
end

kossak_obj (generic function with 1 method)

In [4]:
# Define initial parameters
initial_params = [0.0, 0.0, 0.0,  # H parameters: ϵ, h_Re, h_Im
                  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,  # m_Re parameters
                  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]; # m_Im parameters

initial_params .+= 1;

In [5]:
# Define the objective function wrapper
function objective(params)

    objGEXY = kossak_obj(params, ρᵍ, tᵍ) + kossak_obj(params, ρᵉ, tᵉ) + kossak_obj(params, ρˣ, tˣ) + kossak_obj(params, ρʸ, tʸ)

    return objGEXY
end

objective (generic function with 1 method)

In [6]:
# Define the density matrix evolution and time points

file_name = "../DATA/ALL_GAMMAS_B4_D10.h5"

#γ = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133", "79.477", "251.33"]

γ = "0.25133"
γᶠ = parse(ComplexF64, γ)

tᵍ, ρᵍ = read_timeevolution(file_name, "B1", γ)
tᵉ, ρᵉ = read_timeevolution(file_name, "B2", γ)
tˣ, ρˣ = read_timeevolution(file_name, "B3", γ)
tʸ, ρʸ = read_timeevolution(file_name, "B4", γ);

In [10]:
search_range = (-25.0, 25.0)

# List of methods to compare
methods = [
    :adaptive_de_rand_1_bin_radiuslimited,
    :adaptive_de_rand_1_bin,
    :de_rand_1_bin_radiuslimited,
    :de_rand_1_bin,
    :de_best_1_bin,
    :de_rand_2_bin,
    :de_best_2_bin,
    :pbasis_de_rand_2_bin_radiuslimited,
    :pbasis_de_rand_2_bin,
    :adaptive_de_rand_2_bin_radiuslimited,
    :adaptive_de_rand_2_bin,
    :sep_de_rand_1_bin,
    :sep_de_rand_2_bin,
    :sep_de_best_1_bin,
    :sep_de_best_2_bin,
    :multi_strategy_adaptive_de,
    :multi_de_rand_1_bin,
    :multi_de_best_1_bin,
    :multi_de_rand_2_bin,
    :multi_de_best_2_bin,
    :best_1_bin_radiuslimited,
    :rand_1_bin_radiuslimited,
    :rand_2_bin_radiuslimited,
    :best_2_bin_radiuslimited,
    :best_1_bin,
    :rand_1_bin,
    :rand_2_bin,
    :best_2_bin,
    :self_adaptive_de_rand_1_bin,
    :self_adaptive_de_rand_2_bin,
    :self_adaptive_de_best_1_bin,
    :self_adaptive_de_best_2_bin,
    :gradient_de_rand_1_bin,
    :gradient_de_rand_2_bin,
    :gradient_de_best_1_bin,
    :gradient_de_best_2_bin
]

36-element Vector{Symbol}:
 :adaptive_de_rand_1_bin_radiuslimited
 :adaptive_de_rand_1_bin
 :de_rand_1_bin_radiuslimited
 :de_rand_1_bin
 :de_best_1_bin
 :de_rand_2_bin
 :de_best_2_bin
 :pbasis_de_rand_2_bin_radiuslimited
 :pbasis_de_rand_2_bin
 :adaptive_de_rand_2_bin_radiuslimited
 ⋮
 :best_2_bin
 :self_adaptive_de_rand_1_bin
 :self_adaptive_de_rand_2_bin
 :self_adaptive_de_best_1_bin
 :self_adaptive_de_best_2_bin
 :gradient_de_rand_1_bin
 :gradient_de_rand_2_bin
 :gradient_de_best_1_bin
 :gradient_de_best_2_bin

In [12]:
res_compare = compare_optimizers(objective; SearchRange = (-26, 26),
 NumDimensions =length(initial_params),
 methods=methods,
 max_steps=50000,
 autodiff = :forward
 #MaxTime = 25.0
 );

Starting optimization with optimizer DiffEvoOpt{FitPopulation{Float64}, SimpleSelector, BlackBoxOptim.AdaptiveDiffEvoRandBin{3}, RandomBound{ContinuousRectSearchSpace}}
0.00 secs, 0 evals, 0 steps
0.74 secs, 4 evals, 2 steps, improv/step: 1.000 (last = 1.0000), fitness=51404.727113512
1.49 secs, 8 evals, 4 steps, improv/step: 0.750 (last = 0.5000), fitness=46755.576860885
2.22 secs, 12 evals, 6 steps, improv/step: 0.833 (last = 1.0000), fitness=46755.576860885
2.95 secs, 16 evals, 8 steps, improv/step: 0.875 (last = 1.0000), fitness=46755.576860885
3.67 secs, 20 evals, 10 steps, improv/step: 0.800 (last = 0.5000), fitness=46755.576860885
4.40 secs, 24 evals, 12 steps, improv/step: 0.667 (last = 0.0000), fitness=27366.169105130
5.13 secs, 28 evals, 14 steps, improv/step: 0.643 (last = 0.5000), fitness=27366.169105130
5.85 secs, 32 evals, 16 steps, improv/step: 0.688 (last = 1.0000), fitness=25253.976845701
6.59 secs, 36 evals, 18 steps, improv/step: 0.667 (last = 0.5000), fitness=25253.

Excessive output truncated after 524326 bytes.

In [ ]:
# Extract optimized parameters
optimized_params = best_candidate(res_compare)

# Substitute the optimized parameters back into H and C
optimized_H = construct_H(optimized_params)
optimized_C = construct_C(optimized_params)

println("Optimized H: ", optimized_H)
println("Optimized C: ", optimized_C)

In [ ]:
optimized_H

In [ ]:
optimized_C

In [ ]:
# Setup the BlackBoxOptim optimizer
res = bboptimize(objective, SearchRange = (-25.0, 25.0), NumDimensions = length(initial_params), Method = :adaptive_de_rand_1_bin_radiuslimited, MaxSteps = 10000)

# Extract optimized parameters
optimized_params = best_candidate(res)

# Substitute the optimized parameters back into H and C
optimized_H = construct_H(optimized_params)
optimized_C = construct_C(optimized_params)

println("Optimized H: ", optimized_H)
println("Optimized C: ", optimized_C)

In [8]:
optimized_H

2×2 Matrix{ComplexF64}:
 12.2714+0.0im       1.34831+9.72057im
 1.34831-9.72057im  -12.2714+0.0im

In [9]:
optimized_C

3×3 Matrix{ComplexF64}:
  2454.08+0.0im       38.6602+2449.37im  -76.4486-15.0218im
  38.6602-2449.37im   2446.43+0.0im      -19.7415+78.9891im
 -76.4486+15.0218im  -19.7415-78.9891im   27.3426+0.0im

In [ ]:
result_bbo = bboptimize(polyGEXY; SearchRange = (-26.0, 26.0), NumDimensions = 21,
    MaxTime = 20, 
    Method = :adaptive_de_rand_1_bin_radiuslimited,
    InitialPopulation = [initial_guess])

optimal_value_bbo = best_candidate(result_bbo)
minimum_function_value_bbo = best_fitness(result_bbo)